## Inference with NCES

In [2]:
from ontolearn.concept_learner import NCES
from ontolearn.knowledge_base import KnowledgeBase
from owlapy.parser import DLSyntaxParser
import random

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [7]:
nces = NCES(knowledge_base_path="./NCESData/mutagenesis/mutagenesis.owl", learner_name="SetTransformer",
     path_of_embeddings="./NCESData/mutagenesis/embeddings/ConEx_entity_embeddings.csv", load_pretrained=False, max_length=48, proj_dim=128, rnn_n_layers=2, drop_prob=0.1, num_heads=4, num_seeds=1, num_inds=32, pretrained_model_name="SetTransformer")

In [4]:
KB = KnowledgeBase(path=nces.knowledge_base_path)

In [4]:
dl_parser = DLSyntaxParser(nces.kb_namespace)

### Let's learn the concept ``Father''

In [5]:
father = dl_parser.parse('Father')

In [6]:
not_father = dl_parser.parse('¬Father') # For negative examples

In [7]:
pos = [ind.get_iri().as_str().split("/")[-1] for ind in KB.individuals(father)]

In [8]:
neg = [ind.get_iri().as_str().split("/")[-1] for ind in KB.individuals(not_father)]

In [26]:
## NCES uses a given number of positive and negative examples.

pos_sample = pos[:nces.num_examples]
neg_sample = neg[:nces.num_examples]

In [27]:
concept = nces.fit(pos_sample, neg_sample)

Prediction:  Father


In [39]:
concept

OWLClass(IRI('http://www.benchmark.org/family#','Father'))

### Scalability of NCES (solving multiple learning problems in a go!)

In [19]:
import json, time

In [17]:
with open("../Ontolearn/NCESData/family/training_data/Data.json") as file:
    data = list(json.load(file).items())
## The function below takes an iterable of tuples
LPs = list(map(lambda x: (x[0], x[1]["positive examples"], x[1]["negative examples"]), data))

In [18]:
## We solve 256 learning problems!
lps = LPs[:256]

In [23]:
t0 = time.time()
concepts = nces.fit_from_iterable(lps, verbose=False) ## Now predict with nces.fit_from_iterable.
t1 = time.time()
print("Duration: ", t1-t0, "seconds!")

Duration:  1.2851624488830566 seconds!


In [22]:
print(concepts[0])

OWLObjectUnionOf((OWLObjectIntersectionOf((OWLClass(IRI('http://www.benchmark.org/family#','PersonWithASibling')), OWLObjectAllValuesFrom(property=OWLObjectProperty(IRI('http://www.benchmark.org/family#','hasSibling')),filler=OWLObjectComplementOf(OWLClass(IRI('http://www.benchmark.org/family#','Grandparent')))))), OWLObjectSomeValuesFrom(property=OWLObjectProperty(IRI('http://www.benchmark.org/family#','hasChild')),filler=OWLObjectComplementOf(OWLClass(IRI('http://www.benchmark.org/family#','Son'))))))

In [25]:
print(len(concepts))

256


### Change pretrained model name, e.g., use ensemble model prediction

In [28]:
nces.pretrained_model_name = ['SetTransformer', 'GRU']

In [29]:
nces.pretrained_model_name

['SetTransformer', 'GRU']

In [30]:
nces.refresh()

In [33]:
t0 = time.time()
concepts = nces.fit_from_iterable(lps, verbose=False)
t1 = time.time()
print("Duration: ", t1-t0, "seconds!")

Duration:  3.037950038909912 seconds!


In [34]:
concepts[0]

OWLObjectUnionOf((OWLObjectIntersectionOf((OWLClass(IRI('http://www.benchmark.org/family#','PersonWithASibling')), OWLObjectSomeValuesFrom(property=OWLObjectProperty(IRI('http://www.benchmark.org/family#','hasSibling')),filler=OWLObjectComplementOf(OWLClass(IRI('http://www.benchmark.org/family#','Grandparent')))))), OWLObjectSomeValuesFrom(property=OWLObjectProperty(IRI('http://www.benchmark.org/family#','hasChild')),filler=OWLObjectComplementOf(OWLClass(IRI('http://www.benchmark.org/family#','Son'))))))

## Training NCES

In [5]:
import json
with open("../Ontolearn/NCESData/family/training_data/Data.json") as file:
    data = json.load(file)

In [6]:
from ontolearn.concept_learner import NCES

In [8]:
nces = NCES(knowledge_base_path="./NCESData/family/family.owl", learner_name="SetTransformer",
     path_of_embeddings="./NCESData/family/embeddings/ConEx_entity_embeddings.csv", max_length=48, proj_dim=128, rnn_n_layers=2, drop_prob=0.1, num_heads=4, num_seeds=1, num_inds=32,
            load_pretrained=False, pretrained_model_name="SetTransformer")

In [10]:
nces.train(list(data.items())[-200:], epochs=10, learning_rate=0.001, save_model=True)

********************Trainable model size********************
Synthesizer:  515296
********************Trainable model size********************

Training on CPU, it may take long...

##################################################

SetTransformer starts training... 

################################################## 



Loss: 1.6630, Soft Acc: 46.98%, Hard Acc: 26.53%: 100%|██████████| 10/10 [00:40<00:00,  4.01s/it]

Top performance: loss: 1.6630, soft accuracy: 47.53 ... hard accuracy: 26.53

SetTransformer saved
